In [19]:
# objętość prostaty i jej stref
# objętość zmian na sekwencjach, gdzie zostały obrysowane (niezależnie T2W, ADC, DWI, DCE)
# średnia wartość ADC dla zmian
# mapowanie obrysów prostaty (strefa PZ, TZ) na ADC i obliczenie średniego ADC w każdej z nich - do obliczenia średniego ADC należy pominąć obszary zmian
# obwód prostaty mierzony na warstwie, gdzie maska prostaty ma największą powierzchnię

#good example to test registration
# https://github.com/InsightSoftwareConsortium/ITKElastix/blob/main/examples/ITK_Example08_SimpleTransformix.ipynb

In [20]:
import numpy as np
import SimpleITK as sitk
import mdai
import pandas as pd
import numpy as np
import cv2
import pydicom
import os
import multiprocessing as mp
import functools
from functools import partial
import mdai
import math
import time
import mainFuncs
import itertools
import torch
from os import path as pathOs
import more_itertools
from os.path import basename, dirname, exists, isdir, join, split
import shutil
from mainFuncs import getLabelsAbbrev
import os
from radiomics import firstorder, getTestCase, glcm, glrlm, glszm, imageoperations, shape,shape2D
import SimpleITK as sitk
import mdai
import pandas as pd
import numpy as np
import cv2
import pydicom
import os
import multiprocessing as mp
import functools
from functools import partial
import mdai
import math
import time
import itertools
from pydicom.fileset import FileSet
from os import path as pathOs
from pathlib import Path
import toolz
from toolz.curried import pipe, map, filter, get
from toolz import curry
from os.path import basename, dirname, exists, isdir, join, split
import nnunetv2

import nnunet.elastixRegister as elastixRegister
from nnunet.elastixRegister import reg_a_to_b,reg_a_to_b_itk
import tempfile
import shutil
import re
from toolz.itertoolz import groupby
from toolz import curry
# import multiprocess

In [21]:
import pandas as pd
ihor_csv_dir= '/home/sliceruser/workspaces/konwersjaJsonData/metaDataProject/SR_per_radiologist_full.csv'
prost_volumes_calculated= '/workspaces/konwersjaJsonData/explore/prost_volumes.csv'
ihor_csv_df=pd.read_csv(ihor_csv_dir)
prost_volumes_calculated_df= pd.read_csv(prost_volumes_calculated)
prost_volumes_calculated_df['patient_id']=prost_volumes_calculated_df['number']
prost_volumes_calculated_df['patient_id'] = prost_volumes_calculated_df['patient_id'].astype(float)
prost_volumes_calculated_df


,Unnamed: 0,study_id,number,volume,name,patient_id
0,0,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,36.249082,pg,234.0
1,1,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,0.765703,afs,234.0
2,2,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,2.323213,cz,234.0
3,3,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,8.654765,pz,234.0
4,4,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,12.425273,sv,234.0
...,...,...,...,...,...,...
2463,2463,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,17.440049,pz,1188.0
2464,2464,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,10.835859,sv,1188.0
2465,2465,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,10.714043,sv,1188.0
2466,2466,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,17.585068,tz,1188.0


In [22]:
prost_volumes_calculated_df_calc=prost_volumes_calculated_df[['patient_id', 'volume', 'name']]
prost_volumes_calculated_df_calc=prost_volumes_calculated_df_calc.loc[prost_volumes_calculated_df_calc['name'] == 'pg'][['patient_id', 'volume']]
prost_volumes_calculated_df_calc=prost_volumes_calculated_df_calc.sort_values(by=['patient_id'])
ihor_csv_df=ihor_csv_df.sort_values(by=['patient_id'])
ihor_csv_df


,_id,adequate_fov_dce,adequate_fov_dwi,adequate_fov_t2w,adequate_resolution_dce,adequate_resolution_dwi,adequate_resolution_t2w,adequate_slice_thickness_dce,adequate_slice_thickness_dwi,adequate_slice_thickness_t2w,...,total_time_dce,uniformity_pz,visual_assessment_dce,visual_assessment_dwi,visual_assessment_t2w,visual_params_dce,visual_params_dwi,visual_params_t2w,xnat_project_id,zaxis_t2w
312,62b412649953680001e68e71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,HETEROGENEOUS,NaN,NaN,NaN,NaN,NaN,NaN,AI4AR_PCa,NaN
492,62d2cd8099536800013e1a4d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,HETEROGENEOUS,NaN,NaN,NaN,NaN,NaN,NaN,AI4AR_PCa,NaN
1244,63452e6899536800013e8ae0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,HETEROGENEOUS,NaN,NaN,NaN,NaN,NaN,NaN,AI4AR_PCa,NaN
0,628a95d79953680001d319a6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,HETEROGENEOUS,NaN,NaN,NaN,NaN,NaN,NaN,AI4AR_PCa,NaN
20,6295cc2299536800011be602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,HETEROGENEOUS,NaN,NaN,NaN,NaN,NaN,NaN,AI4AR_PCa,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1391,635eb1d099536800013ea89c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,HETEROGENEOUS,NaN,NaN,NaN,NaN,NaN,NaN,AI4AR_PCa,NaN
1315,635046d399536800013e96af,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,HETEROGENEOUS,NaN,NaN,NaN,NaN,NaN,NaN,AI4AR_PCa,NaN
1471,637228d399536800013ebc16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,HETEROGENEOUS,NaN,NaN,NaN,NaN,NaN,NaN,AI4AR_PCa,NaN
1316,63504aeb99536800013e96bd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,HETEROGENEOUS,NaN,NaN,NaN,NaN,NaN,NaN,AI4AR_PCa,NaN


In [23]:
a=ihor_csv_df[['patient_id','prostate_volume_calculated']].drop_duplicates(subset=['patient_id']).sort_values(by=['patient_id'])

prost_volumes_calculated_df

,Unnamed: 0,study_id,number,volume,name,patient_id
0,0,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,36.249082,pg,234.0
1,1,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,0.765703,afs,234.0
2,2,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,2.323213,cz,234.0
3,3,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,8.654765,pz,234.0
4,4,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,12.425273,sv,234.0
...,...,...,...,...,...,...
2463,2463,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,17.440049,pz,1188.0
2464,2464,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,10.835859,sv,1188.0
2465,2465,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,10.714043,sv,1188.0
2466,2466,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,17.585068,tz,1188.0


In [24]:
b=prost_volumes_calculated_df_calc
merged=pd.merge(a, b, on=["patient_id"])
diffs=merged['prostate_volume_calculated'].to_numpy()-merged['volume'].to_numpy()
diffs=np.abs(diffs)
mean= np.round(np.mean(diffs),2)
var = np.round(np.std(diffs),2)
print(f"mean {mean} var {var} mean value {np.mean(merged['volume'])} ")

mean 8.62 var 8.11 mean value 55.48005456493448 


In [25]:
np.unique(prost_volumes_calculated_df['name'])

array(['afs', 'cz', 'pg', 'pz', 'sv', 'tz', 'ur'], dtype=object)

## check if parts of the prostate is always smaller then full prostate

In [26]:
# objętość prostaty i jej stref
# check if parts of the prostate is always smaller then full prostate
def groupByMaster(rowws):
    grouped_by_master= groupby(lambda row : row[1]['patient_id'],rowws)
    return dict(grouped_by_master).items()

def get_vol(row,colName):
    if(  row[1]['name'] ==colName ):
        return row[1]['volume']
    return 0.0

def get_vols(colName,listRows):
    """
    extracting all paths of intrest from row
    """
    res=  list(map( partial(get_vol,colName=colName),listRows ))
    return np.max(res)


def iterGroups(groupTuple,anatomy_cols, prostate_col_name):
    masterOlds, listRows= groupTuple
    #first find all of the lesion paths and anatomy paths
    anatomy_vols=toolz.pipe(anatomy_cols
                ,map(partial(get_vols,listRows=listRows))
                ,list) 
    prost_vol= get_vols(prostate_col_name,listRows)
    if(prost_vol==0):
        return (masterOlds,-1.0)
    res= list(map( lambda part_vol : part_vol>prost_vol ,anatomy_vols ))
    return (masterOlds,np.sum(res) )


def are_parts_smaller(sourceFrame,anatomy_cols,prost_name):
    vals= toolz.pipe(sourceFrame.iterrows()
                            ,groupByMaster
                            ,list
                            ,map(partial(iterGroups,anatomy_cols=anatomy_cols,prostate_col_name=prost_name ))
                            ,list                                                        )



    return vals
 
names= ['afs', 'cz', 'pz', 'sv', 'tz', 'ur']
prost_name= 'pg'
bi=are_parts_smaller(prost_volumes_calculated_df,names,prost_name )
values=list(map(lambda el: el[1] ,bi))
np.sum(values)#if sum is 0 we have no cases of part bigger then full prostate 

0

## check is lesion dims and lesion volumes are proportional

In [27]:
# objętość zmian na sekwencjach, gdzie zostały obrysowane (niezależnie T2W, ADC, DWI, DCE)
#lesion_dim_short	lesion_dim_vertical
ihor_for_lesion_check_dir = '/home/sliceruser/workspaces/konwersjaJsonData/metaDataProject/AI4AR_PCa_radiological_full.csv'
ihor_for_lesion_check = pd.read_csv(ihor_for_lesion_check_dir)
ihor_for_lesion_check=ihor_for_lesion_check[['patient_id', 'lesion_id', 'rad_hash', 'lesion_dim_long','lesion_dim_max_calculated','lesion_dim_short','lesion_dim_vertical']]
ihor_for_lesion_check



# merged=pd.merge(a, b, on=["patient_id"])

,patient_id,lesion_id,rad_hash,lesion_dim_long,lesion_dim_max_calculated,lesion_dim_short,lesion_dim_vertical
0,1,1,U_OjmNZ8,0.0,16.0,16,0.0
1,1,1,U_8ZmM76,0.0,29.0,29,0.0
2,1,1,U_OrjnrO,31.0,31.0,11,20.0
3,3,1,U_8ekGLQ,0.0,9.0,9,0.0
4,3,1,U_OjmNZ8,0.0,7.0,7,0.0
...,...,...,...,...,...,...,...
1063,3359,1,U_8ekGLQ,0.0,20.0,20,0.0
1064,3359,1,U_8ZmM76,0.0,17.0,17,0.0
1065,3360,1,U_8ekGLQ,0.0,20.0,20,0.0
1066,3360,1,U_OjmNZ8,0.0,17.0,17,0.0


In [28]:
lesion_volumes_dir='/workspaces/konwersjaJsonData/explore/lesion_volumes.csv'
lesion_volumes = pd.read_csv(lesion_volumes_dir)
lesion_volumes['patient_id']=lesion_volumes['number']
lesion_volumes['rad_hash']=lesion_volumes['annotator_id']
lesion_volumes['lesion_id']=list(map(lambda el: int(el.replace('lesion', '')) ,lesion_volumes['lesion_name'].to_numpy()))
lesion_volumes


,Unnamed: 0,study_id,number,modality,volume,lesion_name,annotator_id,patient_id,rad_hash,lesion_id
0,0,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,t2w,2.413125,lesion1,U_8ZmM76,234,U_8ZmM76,1
1,1,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,t2w,2.839482,lesion1,U_8ekGLQ,234,U_8ekGLQ,1
2,2,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,t2w,2.166592,lesion1,U_OjmNZ8,234,U_OjmNZ8,1
3,3,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,t2w,0.846914,lesion2,U_8ZmM76,234,U_8ZmM76,2
4,4,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,t2w,0.522070,lesion2,U_8ekGLQ,234,U_8ekGLQ,2
...,...,...,...,...,...,...,...,...,...,...
3981,3981,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,adc,0.760860,lesion1,U_8ZmM76,1188,U_8ZmM76,1
3982,3982,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,adc,1.299802,lesion1,U_8ekGLQ,1188,U_8ekGLQ,1
3983,3983,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,adc,0.412132,lesion1,U_OjmNZ8,1188,U_OjmNZ8,1
3984,3984,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,adc,1.363207,lesion2,U_8ekGLQ,1188,U_8ekGLQ,2


In [29]:
merged=pd.merge(lesion_volumes, ihor_for_lesion_check, on=["patient_id", "rad_hash","lesion_id"])
mergedd=merged[['lesion_dim_long','lesion_dim_max_calculated','lesion_dim_short','lesion_dim_vertical',"volume"]]
mergedd.corr(method ='pearson')

,lesion_dim_long,lesion_dim_max_calculated,lesion_dim_short,lesion_dim_vertical,volume
lesion_dim_long,1.000000,0.206714,0.125640,0.893632,0.282208
lesion_dim_max_calculated,0.206714,1.000000,0.990331,0.230209,0.594372
lesion_dim_short,0.125640,0.990331,1.000000,0.158530,0.581229
lesion_dim_vertical,0.893632,0.230209,0.158530,1.000000,0.317843
volume,0.282208,0.594372,0.581229,0.317843,1.000000


In [30]:
from scipy.stats import pearsonr
import pandas as pd

def calculate_pvalues(df):
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            tmp = df[df[r].notnull() & df[c].notnull()]
            pvalues[r][c] = round(pearsonr(tmp[r], tmp[c])[1], 4)
    return pvalues

calculate_pvalues(mergedd)

,lesion_dim_long,lesion_dim_max_calculated,lesion_dim_short,lesion_dim_vertical,volume
lesion_dim_long,0.0,0.0,0.0,0.0,0.0
lesion_dim_max_calculated,0.0,0.0,0.0,0.0,0.0
lesion_dim_short,0.0,0.0,0.0,0.0,0.0
lesion_dim_vertical,0.0,0.0,0.0,0.0,0.0
volume,0.0,0.0,0.0,0.0,0.0


## check is adc  related to pirads

In [31]:
def string_pirads_to_int(el):
    try:
        return int(el)
    except:
        return -1    

In [32]:
mean_adcs_dir='/workspaces/konwersjaJsonData/explore/mean_adcs.csv'
mean_adcs_lesions_df = pd.read_csv(mean_adcs_dir)
'lesion' in mean_adcs_lesions_df['lesion_name']
to_filter=list(map(lambda name: 'lesion' in name,mean_adcs_lesions_df['lesion_name'].to_numpy() ))
mean_adcs_lesions_df=mean_adcs_lesions_df[to_filter]
mean_adcs_lesions_df['patient_id']=mean_adcs_lesions_df['number']
mean_adcs_lesions_df['rad_hash']=mean_adcs_lesions_df['annotator_id']
mean_adcs_lesions_df['lesion_id']=list(map(lambda el: int(el.replace('lesion', '')) ,mean_adcs_lesions_df['lesion_name'].to_numpy()))


ihor_for_lesion_check = pd.read_csv(ihor_for_lesion_check_dir)
ihor_for_lesion_check['lesion_pirads']= list(map(string_pirads_to_int,ihor_for_lesion_check['lesion_pirads'].to_numpy() ))
merged=pd.merge(mean_adcs_lesions_df, ihor_for_lesion_check, on=["patient_id", "rad_hash","lesion_id"])
mergedd=merged[['lesion_pirads',"mean_adc"]]
calculate_pvalues(mergedd)
# mergedd

,lesion_pirads,mean_adc
lesion_pirads,0.0,0.0
mean_adc,0.0,0.0


In [33]:
# średnia wartość ADC dla zmian

## check is anatomy mean adc is consistent among anatomic areas and is in adc calculated volume volume of anatomic region is smaller if some lesion is present there

In [34]:
mean_adc_anatomy_dir= '/workspaces/konwersjaJsonData/explore/mean_adcs_per_anatomy.csv'
mean_anatomy_df = pd.read_csv(mean_adc_anatomy_dir)
mean_anatomy_df

,Unnamed: 0,id,pg_noSeg,pg_noSeg_volume,afs_noSeg,afs_noSeg_volume,cz_noSeg,cz_noSeg_volume,pz_noSeg,pz_noSeg_volume,sv_l_noSeg,sv_l_noSeg_volume,sv_r_noSeg,sv_r_noSeg_volume,tz_noSeg,tz_noSeg_volume,ur_noSeg,ur_noSeg_volume
0,0,234,1087.071856,31.765889,927.733333,0.951075,1086.454545,2.789819,1100.365957,7.450084,1562.036745,12.078646,1349.4661458333333,12.173753690473468,1089.7341576506956,20.511506869105038,1194.5862068965516,0.9193720234992983
1,1,666,1158.847477,34.987651,648.517857,2.246913,1184.566667,6.018518,1203.403361,9.549382,1133.641975,6.499999,1044.754716981132,4.25308595781819,1191.5573770491803,17.132714188569498,,
2,2,3,1077.939509,15.135480,149.157895,0.543618,1563.479167,1.373352,979.400000,5.007011,1170.728395,4.635062,1123.04375,4.577838870379596,1134.8081180811807,7.753714586705439,763.6666666666666,1.287517182294261
3,3,17,1250.403809,58.596340,1023.684564,4.263113,1261.675926,3.090041,1278.250000,17.166896,1388.148211,16.794947,1446.2375478927204,14.935199846823512,1266.0541484716157,32.76016058354966,1149.84375,1.8311356134036492
4,4,26,1244.477166,54.147840,880.377778,1.426612,1160.625000,3.043438,1292.114964,17.372960,1400.704327,13.188233,1552.9333333333334,14.741654411071218,1242.8370951913641,32.30482977393886,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,293,1184,1307.521643,128.894778,1233.750000,1.831136,1188.025000,2.288919,1388.028653,9.985411,1405.970037,7.639269,1552.6255707762557,6.2659170863803135,1304.4185814185814,114.56042015464283,1122.375,0.45778389672184944
294,294,1221,1055.123300,58.522475,1014.352941,0.873087,956.715847,4.699260,1230.953020,7.652346,1098.860759,4.057284,1141.4943181818182,4.519506595131743,1036.3067729083666,45.11802890708223,1074.896551724138,0.7446914276069349
295,295,1218,1168.937921,80.655799,858.842105,2.174473,1258.282609,1.316129,1171.273063,15.507429,1038.920635,1.802524,870.6732673267327,2.88976078692712,1177.829797265441,60.68497652546951,1218.4528301886792,1.516409125813241
296,296,1096,1017.364791,78.824663,921.067669,3.805329,987.215385,3.719494,957.265455,7.868161,1112.206897,9.956799,1064.4311594202898,7.896771999923922,1035.9855880985588,61.54332091245056,904.7710843373494,2.374753898527846


In [35]:
print(mean_anatomy_df.mean())
print(mean_anatomy_df.std())


Unnamed: 0            148.500000
id                    882.781879
pg_noSeg             1178.204113
pg_noSeg_volume        52.686436
afs_noSeg            1053.632006
afs_noSeg_volume        2.347886
cz_noSeg             1158.955518
cz_noSeg_volume         3.836004
pz_noSeg             1218.814125
pz_noSeg_volume        12.175264
sv_l_noSeg           1209.972727
sv_l_noSeg_volume       6.989196
dtype: float64
Unnamed: 0            86.169407
id                   916.741548
pg_noSeg             161.163480
pg_noSeg_volume       32.580495
afs_noSeg            299.812454
afs_noSeg_volume       1.913312
cz_noSeg             179.966913
cz_noSeg_volume        3.077398
pz_noSeg             191.580926
pz_noSeg_volume        5.486483
sv_l_noSeg           271.270502
sv_l_noSeg_volume      4.009195
dtype: float64


The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.


In [36]:
prost_volumes_calculated_df

,Unnamed: 0,study_id,number,volume,name,patient_id
0,0,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,36.249082,pg,234.0
1,1,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,0.765703,afs,234.0
2,2,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,2.323213,cz,234.0
3,3,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,8.654765,pz,234.0
4,4,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,12.425273,sv,234.0
...,...,...,...,...,...,...
2463,2463,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,17.440049,pz,1188.0
2464,2464,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,10.835859,sv,1188.0
2465,2465,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,10.714043,sv,1188.0
2466,2466,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,17.585068,tz,1188.0


In [37]:
prost_volumes_calculated_df['patient_id']=prost_volumes_calculated_df['number'] 
mean_anatomy_df['patient_id']=mean_anatomy_df['id'] 

merged=pd.merge(prost_volumes_calculated_df, mean_anatomy_df, on=["patient_id"])
merged

,Unnamed: 0_x,study_id,number,volume,name,patient_id,Unnamed: 0_y,id,pg_noSeg,pg_noSeg_volume,...,pz_noSeg,pz_noSeg_volume,sv_l_noSeg,sv_l_noSeg_volume,sv_r_noSeg,sv_r_noSeg_volume,tz_noSeg,tz_noSeg_volume,ur_noSeg,ur_noSeg_volume
0,0,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,36.249082,pg,234,0,234,1087.071856,31.765889,...,1100.365957,7.450084,1562.036745,12.078646,1349.4661458333333,12.173753690473468,1089.7341576506956,20.511506869105038,1194.5862068965516,0.9193720234992983
1,1,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,0.765703,afs,234,0,234,1087.071856,31.765889,...,1100.365957,7.450084,1562.036745,12.078646,1349.4661458333333,12.173753690473468,1089.7341576506956,20.511506869105038,1194.5862068965516,0.9193720234992983
2,2,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,2.323213,cz,234,0,234,1087.071856,31.765889,...,1100.365957,7.450084,1562.036745,12.078646,1349.4661458333333,12.173753690473468,1089.7341576506956,20.511506869105038,1194.5862068965516,0.9193720234992983
3,3,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,8.654765,pz,234,0,234,1087.071856,31.765889,...,1100.365957,7.450084,1562.036745,12.078646,1349.4661458333333,12.173753690473468,1089.7341576506956,20.511506869105038,1194.5862068965516,0.9193720234992983
4,4,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,12.425273,sv,234,0,234,1087.071856,31.765889,...,1100.365957,7.450084,1562.036745,12.078646,1349.4661458333333,12.173753690473468,1089.7341576506956,20.511506869105038,1194.5862068965516,0.9193720234992983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2282,2463,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,17.440049,pz,1188,297,1188,1146.217466,37.028500,...,1238.871951,15.597622,1222.235616,11.571406,1448.7806267806268,11.127571483825985,1090.7142857142858,16.86572088146844,958.8571428571429,1.109586900096608
2283,2464,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,10.835859,sv,1188,297,1188,1146.217466,37.028500,...,1238.871951,15.597622,1222.235616,11.571406,1448.7806267806268,11.127571483825985,1090.7142857142858,16.86572088146844,958.8571428571429,1.109586900096608
2284,2465,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,10.714043,sv,1188,297,1188,1146.217466,37.028500,...,1238.871951,15.597622,1222.235616,11.571406,1448.7806267806268,11.127571483825985,1090.7142857142858,16.86572088146844,958.8571428571429,1.109586900096608
2285,2466,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,17.585068,tz,1188,297,1188,1146.217466,37.028500,...,1238.871951,15.597622,1222.235616,11.571406,1448.7806267806268,11.127571483825985,1090.7142857142858,16.86572088146844,958.8571428571429,1.109586900096608


In [38]:
def groupByMaster(rowws):
    grouped_by_master= groupby(lambda row : row[1]['patient_id'],rowws)
    return dict(grouped_by_master).items()

def my_to_float(el):
    try: 
        return float(el)
    except:
        return 0.0

def get_vol(row,colName):
    if(  row[1]['name'] ==colName ):
        return row[1]['volume']-my_to_float(row[1][f"{colName}_noSeg_volume"])
    return 0.0

def get_vols(colName,listRows):
    """
    extracting all paths of intrest from row
    """
    res=  list(map( partial(get_vol,colName=colName),listRows ))
    return np.max(res)


def iterGroups(groupTuple,anatomy_cols, prostate_col_name):
    masterOlds, listRows= groupTuple
    #first find all of the lesion paths and anatomy paths
    anatomy_vols=toolz.pipe(anatomy_cols
                ,map(partial(get_vols,listRows=listRows))
                ,list) 
    return (masterOlds,np.mean(anatomy_vols) )


def are_parts_smaller(sourceFrame,anatomy_cols,prost_name):
    vals= toolz.pipe(sourceFrame.iterrows()
                            ,groupByMaster
                            ,list
                            ,map(partial(iterGroups,anatomy_cols=anatomy_cols,prostate_col_name=prost_name ))
                            ,list                                                        )



    return vals
 
names= ['afs', 'cz', 'pz', 'sv_l','sv_r', 'tz', 'ur']
prost_name= 'pg'
bi=are_parts_smaller(merged,names,prost_name )
values=list(map(lambda el: el[1] ,bi))
np.mean(values)#if sum is 0 we have no cases of part bigger then full prostate 

1.0095241380704647

In [39]:
# mapowanie obrysów prostaty (strefa PZ, TZ) na ADC i obliczenie średniego ADC w każdej z nich - do obliczenia średniego ADC należy pominąć obszary zmian


## find is there correletion between volumeand dime from ihor and max perimeters 


In [40]:
perimeters_dir='/workspaces/konwersjaJsonData/explore/max_circs.csv'
perimeters_df = pd.read_csv(perimeters_dir)
perimeters_df['patient_id']=perimeters_df['id']

merged=pd.merge(perimeters_df, ihor_csv_df, on=["patient_id"])

mergedddd=merged[['pg_noSeg','prostate_dim_h','prostate_dim_l',	'prostate_dim_w',	'prostate_volume_approx_ratio',	'prostate_volume_calculated']]
calculate_pvalues(mergedddd)

An input array is constant; the correlation coefficient is not defined.


,pg_noSeg,prostate_dim_h,prostate_dim_l,prostate_dim_w,prostate_volume_approx_ratio,prostate_volume_calculated
pg_noSeg,0.0,0.0,0.0,0.0,NaN,0.0
prostate_dim_h,0.0,0.0,0.0,0.0,NaN,0.0
prostate_dim_l,0.0,0.0,0.0,0.0,NaN,0.0
prostate_dim_w,0.0,0.0,0.0,0.0,NaN,0.0
prostate_volume_approx_ratio,NaN,NaN,NaN,NaN,NaN,NaN
prostate_volume_calculated,0.0,0.0,0.0,0.0,NaN,0.0
